In [4]:
import qdrant_client
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.indices import MultiModalVectorStoreIndex

In [51]:
import yaml

In [57]:
# Assuming the entire 'output.md' file is YAML content
with open('docs/output.md', 'r') as file:
    content = file.read()
    _, front_matter, _ = content.split('---', 2)
    data = yaml.safe_load(front_matter)

In [58]:
print(data)

{'title': 'Release of 4th Quarter 2023 Real Estate Statistics', 'date': datetime.date(2024, 1, 26), 'source_link': 'https://www.ura.gov.sg/Corporate/Media-Room/Media-Releases/pr24-03'}


In [59]:
def get_document_metadata(filepath):
    with open(filepath, 'r') as file:
        content = file.read()
        _, front_matter, _ = content.split('---', 2)
        data = yaml.safe_load(front_matter)
    return data

In [ ]:
# Create a local Qdrant vector store
client = qdrant_client.QdrantClient(path="qdrant_db")

In [68]:
text_store = QdrantVectorStore(
    client=client, collection_name="text_collection"
)
image_store = QdrantVectorStore(
    client=client, collection_name="image_collection"
)
storage_context = StorageContext.from_defaults(
    vector_store=text_store, image_store=image_store
)

In [32]:
client.delete_collection(collection_name="text_collection")
client.delete_collection(collection_name="image_collection")

True

In [64]:
documents = SimpleDirectoryReader(input_dir="test_data", file_metadata=get_document_metadata).load_data()

In [65]:
documents

[Document(id_='bdaa4127-33ac-4282-ab74-a97618825ef0', embedding=None, metadata={'title': 'Release of 4th Quarter 2023 Real Estate Statistics', 'date': '2024-01-26', 'source_link': 'https://www.ura.gov.sg/Corporate/Media-Room/Media-Releases/pr24-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='---\r\ntitle: Release of 4th Quarter 2023 Real Estate Statistics\r\ndate: "2024-01-26"\r\nsource_link: https://www.ura.gov.sg/Corporate/Media-Room/Media-Releases/pr24-03\r\n---\r\n\r', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='70fbf6e0-389d-44e9-8f72-22c7c33b6fb9', embedding=None, metadata={'title': 'Release of 4th Quarter 2023 Real Estate Sta

In [69]:
# Create the MultiModal index
index = MultiModalVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

In [72]:
test_query = "How much did prices of retail spaces increase by in 4th Quarter 2023?"
# generate  retrieval results
retriever = index.as_retriever(similarity_top_k=3)
retrieval_results = retriever.retrieve(test_query)
from llama_index.core.response.notebook_utils import display_source_node
for res_node in retrieval_results:
    display_source_node(res_node, source_length=1000, show_source_metadata=True)

**Node ID:** ff92dcc1-d331-4506-b9f2-e6bde4775e80<br>**Similarity:** 0.9081193335136626<br>**Text:** Prices and Rentals

Prices of retail space increased by 1.2% in 4th Quarter 2023, compared with the 0.6% increase in the previous quarter. Rentals of retail space decreased by 0.1% in 4th Quarter 2023, compared with the 0.5% increase in the previous quarter. For the whole of 2023, prices of retail space increased by 1.2%, compared with the decrease of 7.8% in 2022, while rentals of retail space increased by 0.4%, compared with the decrease of 2.4% in 2022.<br>**Metadata:** {'title': 'Release of 4th Quarter 2023 Real Estate Statistics', 'date': '2024-01-26', 'source_link': 'https://www.ura.gov.sg/Corporate/Media-Room/Media-Releases/pr24-03'}<br>

**Node ID:** b9921882-50ce-4a30-b512-740f8e15e13d<br>**Similarity:** 0.8881903134547322<br>**Text:** Prices and Rentals

Prices of office space decreased by 5.9% in 4th Quarter 2023, compared with the 0.8% increase in the previous quarter. Rentals of office space increased by 0.3% in 4th Quarter 2023, compared with the 4.9% increase in the previous quarter. For the whole of 2023, prices of office space decreased by 4.2%, compared with the decrease of 0.1% in 2022, while rentals of office space increased by 13.1%, compared with the increase of 11.7% in 2022.<br>**Metadata:** {'title': 'Release of 4th Quarter 2023 Real Estate Statistics', 'date': '2024-01-26', 'source_link': 'https://www.ura.gov.sg/Corporate/Media-Room/Media-Releases/pr24-03'}<br>

**Node ID:** 993d2917-b9b8-4f56-a9d2-013a1bd2fbbe<br>**Similarity:** 0.8809591841772909<br>**Text:** Stock and Vacancy - Retail Space

The amount of occupied retail space increased by 63,000 sq m (nett) in 4th Quarter 2023, compared with the decrease of 8,000 sq m (nett) in the previous quarter. The stock of retail space increased by 21,000 sq m (nett) in 4th Quarter 2023, compared with the decrease of 23,000 sq m (nett) in the previous quarter. As a result, the island-wide vacancy rate of retail space decreased to 6.5% as at end of 4th Quarter 2023, from 7.2% as at end of the previous quarter.<br>**Metadata:** {'title': 'Release of 4th Quarter 2023 Real Estate Statistics', 'date': '2024-01-26', 'source_link': 'https://www.ura.gov.sg/Corporate/Media-Room/Media-Releases/pr24-03'}<br>

In [26]:
from llama_index.core.response.notebook_utils import display_source_node

**Node ID:** 4faed6e6-846e-451d-bd91-2ae0abeb96e4<br>**Similarity:** 0.8892642149539416<br>**Text:** Prices and Rentals

Prices of retail space increased by 1.2% in 4th Quarter 2023, compared with the 0.6% increase in the previous quarter. Rentals of retail space decreased by 0.1% in 4th Quarter 2023, compared with the 0.5% increase in the previous quarter. For the whole of 2023, prices of retail space increased by 1.2%, compared with the decrease of 7.8% in 2022, while rentals of retail space increased by 0.4%, compared with the decrease of 2.4% in 2022.<br>

**Node ID:** bbf299ec-c143-403e-8fd0-ce503b8c12e3<br>**Similarity:** 0.8660242835723257<br>**Text:** Prices and Rentals

Prices of office space decreased by 5.9% in 4th Quarter 2023, compared with the 0.8% increase in the previous quarter. Rentals of office space increased by 0.3% in 4th Quarter 2023, compared with the 4.9% increase in the previous quarter. For the whole of 2023, prices of office space decreased by 4.2%, compared with the decrease of 0.1% in 2022, while rentals of office space increased by 13.1%, compared with the increase of 11.7% in 2022.<br>

**Node ID:** d0f81c46-cd80-44e6-8549-f52d378e1646<br>**Similarity:** 0.8658311761771362<br>**Text:** Stock and Vacancy - Retail Space

The amount of occupied retail space increased by 63,000 sq m (nett) in 4th Quarter 2023, compared with the decrease of 8,000 sq m (nett) in the previous quarter. The stock of retail space increased by 21,000 sq m (nett) in 4th Quarter 2023, compared with the decrease of 23,000 sq m (nett) in the previous quarter. As a result, the island-wide vacancy rate of retail space decreased to 6.5% as at end of 4th Quarter 2023, from 7.2% as at end of the previous quarter.<br>